In [ ]:
# Load in custom functions to make this class more focused on just
# training models
from utils import load_data, standardize, normalize, replace_nans, split_data, coord_to_index, preview_graph, get_metric_name, get_base_model_mse_loss, get_model_mse_loss, loss_by_metric, loss_by_pixel, distributed_loss

import pandas as pd
import math
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from tensorflow.keras.metrics import mean_squared_error
from keras.layers import Input,Conv2D
from sklearn.model_selection import train_test_split
from scipy import stats
import tensorflow as tf
import gc

In [ ]:
# Load in data
data = load_data("../training_data/*.npy")
print(f"\nLoaded data shape: {data.shape}")

# If normalizing
original_mins = np.nanmin(data, axis=(0,1,2))
original_maxs = np.nanmax(data, axis=(0,1,2))

# Preprocess Data
print("normalize...")
data = normalize(data, wind_special=False)
print("Replacing Nans...")
data = replace_nans(data)
print("Done!")

In [ ]:
# Verify data is normalized and in the expected shape
print(original_mins)
print(original_maxs)
print(np.nanmin(data, axis=(0,1,2)))
print(np.nanmax(data, axis=(0,1,2)))
print(data.shape)

In [ ]:
plt.figure(figsize=(40,20))
pos = plt.imshow(data[180,:,:,2], origin="lower")
plt.colorbar(pos)
plt.show()

In [ ]:
print("Splitting data...")
(x_train_data, y_train_data, x_test_data, y_test_data) = split_data(data, 0.9, days=3, spacing=1)
print("Data Split!")
# auto_encoder_x = data
# auto_encoder_y = data

y_train_data = y_train_data[:,0,:,:]
y_test_data = y_test_data[:,0,:,:]

print(x_train_data.shape)
print(y_train_data.shape)
print(x_test_data.shape)
print(y_test_data.shape)

In [ ]:
def condense(data):
    (sample_count, seq_len, height, width, channels) = data.shape
    data = data.transpose(0,2,3,1,4).reshape(sample_count, height, width, seq_len * channels)
    return data

print(x_train_data.shape)
x_train_data = condense(x_train_data)
print(x_train_data.shape)
x_test_data = condense(x_test_data)

In [ ]:
def gen_positional_encoding(height, width, vec_count = 5):
    
    embedings = []
    
    for i in range(vec_count):
        grid = np.array([np.arange(width) / (width - 1)] * height)
        if i % 2 == 0:
            w_1 = -np.cos(grid * math.pi * (1 + (i * 2)))
        else:
            w_1 = np.sin(grid * math.pi * (1 + (i * 2)))
        w_1 = (w_1 / 2) + 0.5
        embedings.append(w_1)
        
    for i in range(vec_count):
        grid = np.array([np.arange(height) / (height - 1)] * width).T
        if i % 2 == 0:
            w_1 = -np.cos(grid * math.pi * (1 + (i * 2)))
        else:
            w_1 = np.sin(grid * math.pi * (1 + (i * 2)))
        w_1 = (w_1 / 2) + 0.5
        embedings.append(w_1)
    
    return np.array(embedings)

def add_encoding(data, pe):
    encoded_data = []
    for i in range(len(data)):
        element = data[i]
        for i in range(len(pe) - 1, -1, -1):
            element = np.insert(element, 0, pe[1], axis=2)

        encoded_data.append(element)
        
    return np.array(encoded_data)


# plt.imshow(gen_positional_encoding(48, 116)[9])
# plt.colorbar()
# plt.show()

pe = gen_positional_encoding(48, 116)
x_train_data = add_encoding(x_train_data, pe)
x_test_data = add_encoding(x_test_data, pe)

print(x_train_data.shape)
print(x_test_data.shape)

In [ ]:
class DataFeeder(keras.utils.Sequence) :    
    def __init__(self, data_x, data_y, batch_size) :
        self.data_x = data_x
        self.data_y = data_y
        self.batch_size = batch_size
    
    
    def __len__(self) :
        return (np.ceil(self.data_x.shape[0] / float(self.batch_size))).astype(np.int)
    
    
    def __getitem__(self, idx) :
        batch_x = self.data_x[idx * self.batch_size : (idx+1) * self.batch_size]
        batch_y = self.data_y[idx * self.batch_size : (idx+1) * self.batch_size]

        return np.array(batch_x), np.array(batch_y)

In [ ]:
mask = pd.read_csv("./mask.csv", header=None)
mask = mask.to_numpy()
mask = np.stack([mask] * 8, axis=2)

def custom_loss(yTrue,yPred):
    if yTrue.shape[0] == None:
        return mean_squared_error(yTrue, yPred)
    msk = np.stack([mask] * yTrue.shape[0])
    yPred = yPred * msk
    return mean_squared_error(yTrue, yPred)

In [ ]:
keras.backend.clear_session()
gc.collect()
try: 
    del model
except:
    pass

BATCH_SIZE = 16

def create_model():

    model = Sequential()
    model.add(Conv2D(512, (1, 41), activation='sigmoid', padding="same"))
    model.add(Conv2D(512, (41, 1), activation='sigmoid', padding="same"))
    model.add(Conv2D(32,  (3, 3),  activation='sigmoid', padding="same"))
    model.add(Conv2D(512, (1, 9),  activation='sigmoid', padding="same"))
    model.add(Conv2D(512, (9, 1),  activation='sigmoid', padding="same"))
    model.add(Conv2D(32,  (3, 3),  activation='sigmoid', padding="same"))
    model.add(Conv2D(512, (1, 5),  activation='sigmoid', padding="same"))
    model.add(Conv2D(512, (5, 1),  activation='sigmoid', padding="same"))
    model.add(Conv2D(512, (3, 3),  activation='sigmoid', padding="same"))
    model.add(Conv2D(8,   (1, 1),  activation=None, padding="same"))
    
    
    opt = keras.optimizers.Adam(learning_rate=0.0001)

    model.compile(optimizer=opt,
              loss=custom_loss,
              metrics=["mae"])

    return model

print("Compiling...")

model = create_model()
model.build([None, 48, 116, 34])
model.load_weights("./models.h5")

checkpoint = ModelCheckpoint(filepath=f"./model.h5", 
                         monitor='val_loss',
                         verbose=1, 
                         save_best_only=True,
                         mode='min')

print("Fitting...")
model.fit(DataFeeder(x_train_data, y_train_data, BATCH_SIZE), 
          validation_data=DataFeeder(x_test_data, y_test_data, BATCH_SIZE), 
          callbacks=[checkpoint],
          epochs = 100,
          shuffle = True)

model.summary()

In [ ]:
test_input = x_test_data[0:1]
test_output = model.predict(test_input)

print(test_input.shape)
print(test_output.shape)
print(x_test_data.shape)
print(y_test_data.shape)

plt.figure(figsize=(40,20))
pos = plt.imshow(test_output[0,:,:,2], origin="lower")
plt.colorbar(pos)
plt.show()

In [ ]:
bm_loss = get_base_model_mse_loss(x_test_data[:,:,:,-8:], y_test_data)
m_loss = get_model_mse_loss(model, x_test_data, y_test_data)

print(f"Base Model Loss {bm_loss}")
print(f"Model Loss {m_loss}")
print(f"{round(100 * (bm_loss - m_loss) / bm_loss, 2)}% better than baseline model")

losses_by_metric = loss_by_metric(model, x_test_data, y_test_data)
for i in range(8):
    print(f"{get_metric_name(i)} Error : {math.sqrt(losses_by_metric[i]) * (original_maxs[i] - original_mins[i])}")

spatial_loss = loss_by_pixel(model, x_test_data, y_test_data)
plt.figure(figsize=(40,20))
pos = plt.imshow(spatial_loss, origin="lower",  cmap="gray")
plt.colorbar(pos)
plt.show()